In [22]:
import arcpy
import os
from arcpy import analysis as AN
from arcpy import management as DM
from arcpy.sa import *
env = arcpy.env
env.overwriteOutput = True
env.workspace = os.getcwd() + "\\" "ExpectedAccidents.gdb"

In [74]:
print(f"Feature classes: {arcpy.ListFeatureClasses()}")
print(f"Tables: {arcpy.ListTables()}")

Feature classes: ['transbase_collisions_XY', 'SFIntersections_Voronoi_Collisions', 'SFIntersections_Voronoi_Collisions_Join', 'tempfc', 'tempfc0', 'test2', 'SFIntersections_Voronoi_Clipped', 'test1', 'broadsides_collisions', 'headons_collisions', 'merged']
Tables: []


In [8]:
albers3310 = arcpy.SpatialReference(3310)
DM.Project('BayAreaCounties.shp', 'BayAreaCounties3310.shp', albers3310)
sf_county = DM.SelectLayerByAttribute('BayAreaCounties3310.shp', "NEW_SELECTION", '"county" = \'San Francisco\'')
DM.CopyFeatures(sf_county, "sf_county.shp")
DM.MultipartToSinglepart("sf_county.shp", "sf_county_polygons.shp")

Traceback (most recent call last):
  File "c:\program files\arcgis\pro\Resources\ArcToolbox\scripts\calculategeometry.py", line 289, in <module>
    CalculateGeometry(fc, properties, lUnit, aUnit, cs, cformat)
  File "c:\program files\arcgis\pro\Resources\ArcToolbox\scripts\calculategeometry.py", line 127, in CalculateGeometry
    geomValue = eval(geomCalcs[calcProp[1].upper()])
KeyError: '#'


ExecuteError: ERROR 000582: Error occurred during execution.


In [14]:
arcpy.AddField_management("sf_county_polygons.shp", "Shape_area", "DOUBLE")
with arcpy.da.UpdateCursor("sf_county_polygons.shp", ["SHAPE@AREA","Shape_area"]) as cursor:
    for row in cursor:
        row[1] = row[0]
        cursor.updateRow(row)
del row, cursor 

In [26]:
areas = []
with arcpy.da.SearchCursor("sf_county_polygons.shp", ["Shape_area"]) as cursor:
    for row in cursor:
        areas.append(row[0])
del row, cursor 
sql = f'"Shape_area" = {max(areas)}'
sf_city = DM.SelectLayerByAttribute("sf_county_polygons.shp", "NEW_SELECTION", sql)
DM.CopyFeatures(sf_city, "sf_city_boundary.shp")
env.extent = arcpy.Describe("sf_city_boundary.shp").extent

In [32]:
DM.XYTableToPoint("transbase_collisions.csv", "collisions", "longitude", "latitude")
DM.Project("collisions", "collisions3310.shp", albers3310)

<Result 'C:\\Users\\918831919\\Documents\\ArcGIS\\Projects\\ExpectedAccidents\\collisions3310.shp'>

In [7]:
# DM.UnsplitLine("SFSpeedLimits", "streets_unsplit", ["from_st", "to_st"])
# AN.Intersect("streets_unsplit", "intersections", output_type = "POINT")

<Result 'C:\\Users\\918831919\\Documents\\ArcGIS\\Projects\\ExpectedAccidents\\intersections.shp'>

In [30]:
DM.Project("SFIntersections.shp", "SFIntersections3310.shp", albers3310)
AN.CreateThiessenPolygons("SFIntersections3310", "SFIntersections_Voronoi.shp", "ALL")
AN.Clip("SFIntersections_Voronoi.shp", "sf_city_boundary.shp", "SFIntersections_Voronoi_Clipped.shp")

<Result 'C:\\Users\\918831919\\Documents\\ArcGIS\\Projects\\ExpectedAccidents\\SFIntersections_Voronoi_Clipped.shp'>

In [39]:
AN.SummarizeWithin("SFIntersections_Voronoi_Clipped.shp", "collisions.shp", "SFIntersections_Voronoi_Collisions")
DM.AlterField("SFIntersections_Voronoi_Collisions", "Point_Count", "Total_Collisions", "Total_Collisions")

id,value
0,C:\Users\918831919\Documents\ArcGIS\Projects\ExpectedAccidents\ExpectedAccidents.gdb\SFIntersections_Voronoi_Collisions
1,


In [54]:
DM.FeatureToPoint("SFIntersections_Voronoi_Collisions", "Intersection_Points_With_Counts.shp", "CENTROID")

<Result 'C:\\Users\\918831919\\Documents\\ArcGIS\\Projects\\ExpectedAccidents\\Intersection_Points_With_Counts.shp'>

In [ ]:
AN.SummarizeWithin("SFIntersections_Voronoi_Clipped.shp", "collisions.shp", "SFIntersections_Voronoi_Collisions")

In [62]:
def SelectAggregateWrite(selectionFeature, polygonFeature, sel, fieldName, outPolygon):
    try:
        selLayer = DM.SelectLayerByAttribute(selectionFeature, "NEW_SELECTION", sel)
        nRecords = DM.GetCount(selLayer)
    except arcpy.ExecuteError as e:
        print(e)
    print(f"{nRecords[0]} selected.")
    tmpLyr = 'in_memory\\tmp.lyr'
    DM.MakeFeatureLayer(selLayer, tmpLyr) 
    DM.SelectLayerByAttribute(selectionFeature, "CLEAR_SELECTION")
    AN.SummarizeWithin(polygonFeature, tmpLyr, outPolygon)
    DM.AlterField(outPolygon, 'Point_Count', fieldName, fieldName)
    print(f"{outPolygon} created.")
    

In [78]:
args_list = [
    ['n_broadsides', '"type_of_co" = \'Broadside\'', 'broadsides'],
    ['n_headons', '"type_of_co" = \'Head-On\'', 'headons'],
    ['n_readends', '"type_of_co" = \'Rear End\'', 'rearends'],
    ['n_dark', '"lighting" = \'Dark - Street Lights\'', 'darkness'],
    ['n_wet', '"road_surfa" = \'Wet\'', 'wetness'],
    ['n_proceeding_straight', '"party1_mov" = \'Proceeding Straight\'', 'proceeding_straight'],
    ['n_making_left_turn', '"party1_mov" = \'Making Left Turn\'', 'making_left_turn'],
    ['n_unsafe_speed', '"pcf_viol_c" = \'Unsafe speed for prevailing conditions\'', 'unsafe_speed'],
    ['n_unsafe_turn_or_lane_change', '"pcf_viol_c" = \'Unsafe turn or lane change prohibited\'', 'unsafe_turn']
]

In [79]:
collisions = 'collisions3310.shp'
intersections = 'SFIntersections_Voronoi_Clipped.shp'
for args in args_list:
    SelectAggregateWrite(collisions, intersections, args[1], args[0], args[2])

3409 selected.
broadsides created.
713 selected.
headons created.
2106 selected.
rearends created.
2804 selected.
darkness created.
877 selected.
wetness created.
5187 selected.
proceeding_straight created.
1360 selected.
making_left_turn created.
2073 selected.
unsafe_speed created.
881 selected.
unsafe_turn created.


In [83]:
to_merge = [x[2] for x in args_list]
to_merge.append('SFIntersections_Voronoi_Collisions')
DM.Merge(to_merge, 'merged_collisions.shp')

<Result 'C:\\Users\\918831919\\Documents\\ArcGIS\\Projects\\ExpectedAccidents\\merged_collisions.shp'>